In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [2]:
response = requests.get('https://httpbin.org/user-agent')
user_agent = response.json()['user-agent']
print(user_agent)


python-requests/2.31.0


In [3]:

url_template = "https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_TrendingAutoSuggest_9_0_na_na_na&otracker1=AS_Query_TrendingAutoSuggest_9_0_na_na_na&as-pos=9&as-type=TRENDING&suggestionId=laptops&requestId=f8588429-b2b5-4b88-b717-692d5b0649eb&page={}"
headers = {
    'User-Agent': 'python-requests/2.31.0'
}

def fetch_page(url, headers):
    with requests.Session() as session:
        for j in range(5):  # Retry up to 5 times
            try:
                r = session.get(url, headers=headers)
                if r.status_code == 200:
                    soup = BeautifulSoup(r.text, "html")
                    return soup
                else:
                    print(f"Error: {r.status_code}")
            except requests.exceptions.RequestException as e:
                print(f"Request failed: {e}")
            time.sleep(5)  # Wait for 5 seconds before retrying
    return None


'''soup = fetch_page(url, headers)
if soup:
    print(soup)
else:
    print("Failed to retrieve the page after multiple attempts.")
'''

'soup = fetch_page(url, headers)\nif soup:\n    print(soup)\nelse:\n    print("Failed to retrieve the page after multiple attempts.")\n'

In [4]:
Product_name = []
Prices = []
Description = []
Reviews = []
Discount = []
Original = []
Images = []
categories = ['Processor', 'RAM', 'OS', 'SSD', 'Display', 'Warranty']
for i in range(2, 25):
    url = url_template.format(i)
    soup = fetch_page(url, headers)
    if soup:
        try:
            np = soup.find("a", class_="_9QVEpD").get("href")
            cnp = "https://www.flipkart.com" + np
            print(cnp)
            
            box = soup.find("div", class_= "DOjaWF gdgoEp")
            
            names = box.find_all("div", class_ = "KzDlHZ")
            for i in names:
                name = i.text
                Product_name.append(name)
            
            print(len(Product_name))
            
            images = box.find_all("img", class_ = "DByuf4")
            for i in images:
                image = i.get('src')
                Images.append(image)

            print(len(Images))
            
            discount = box.find_all("div", class_ = "UkUFwK")
            for i in discount:
                dis = i.text
                Discount.append(dis)
            
            

            original = box.find_all("div", class_ = "yRaY8j ZYYwLA")
            for i in original:
                orig = i.text
                Original.append(orig)
            
            

            price = box.find_all("div", class_ = "Nx9bqj _4b5DiR")
            for i in price:
                pr = i.text
                Prices.append(pr)
            
            
            
            
            desc = box.find_all("ul", class_ = "G4BRas")
            for i in desc:
                item_dict = {}
                li_items = i.find_all('li')
                for index, li in enumerate(li_items):
                     value = li.text.split('::before')[-1].strip()
                     if index < len(categories):
                         item_dict[categories[index]] = value
                Description.append(item_dict)
            print(len(Description))
            
            ratings = box.find_all("div", class_ = "XQDdHH")
            for i in ratings:
                rev = i.text
                Reviews.append(rev)
                
            max_length = max(len(Product_name), len(Prices), len(Description), len(Reviews), len(Original), len(Discount))
            Reviews.extend([None] * (max_length - len(Reviews)))
            Original.extend([None] * (max_length - len(Original)))
            Discount.extend([None] * (max_length - len(Discount)))
            Prices.extend([None] * (max_length - len(Prices)))
            print(len(Discount))
            print(len(Original))
            print(len(Prices))
            print(len(Reviews))

        
        except AttributeError:
            print(f"No next page link found on page {i}")
    else:
        print(f"Failed to retrieve page {i} after multiple attempts.")


Error: 403
Error: 429
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_TrendingAutoSuggest_9_0_na_na_na&otracker1=AS_Query_TrendingAutoSuggest_9_0_na_na_na&as-pos=9&as-type=TRENDING&suggestionId=laptops&requestId=f8588429-b2b5-4b88-b717-692d5b0649eb&page=1
24
24
24
24
24
24
24
Error: 403
Error: 429
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_TrendingAutoSuggest_9_0_na_na_na&otracker1=AS_Query_TrendingAutoSuggest_9_0_na_na_na&as-pos=9&as-type=TRENDING&suggestionId=laptops&requestId=f8588429-b2b5-4b88-b717-692d5b0649eb&page=2
48
48
48
48
48
48
48
Error: 403
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_TrendingAutoSuggest_9_0_na_na_na&otracker1=AS_Query_TrendingAutoSuggest_9_0_na_na_na&as-pos=9&as-type=TRENDING&suggestionId=laptops&requestId=f8588429-b2b5-4b88-b717-692d5b0649eb&page=3
72
72
72
72
72
72
72
Error: 403
https://www.flipkart.com/search?q=laptops&as=on&as-show=on&otracker=AS_Query_Trend

In [5]:
print(type('Images'))

<class 'str'>


In [6]:
df = pd.DataFrame({"Product Name":Product_name, "Images":Images, "Description":Description, "Original Price":Original,"Discounts":Discount,"Discounted Price":Prices, "Reviews":Reviews})
print(df)
df.to_csv("flipkart_laptops.csv")


                                          Product Name  \
0    Acer Aspire 3 Intel Core i3 12th Gen 1215U - (...   
1    HP 360 Intel Celeron Quad Core N4120 - (4 GB/6...   
2    ASUS Vivobook 15 Intel Core i3 12th Gen 1215U ...   
3    Acer Swift Go 14 EVO OLED (2024) Intel Core i5...   
4    Apple 2022 MacBook AIR Apple M2 - (8 GB/256 GB...   
..                                                 ...   
547  ASUS Vivobook S 15 OLED Intel Core i9 13th Gen...   
548  HP 15q Intel Pentium Quad Core N5000 - (4 GB/1...   
549  HP 247 G8 AMD Ryzen 5 Dual Core 3500U - (8 GB/...   
550  ASUS VivoBook 14 Intel Core i5 10th Gen 10210U...   
551  HP 15s Intel Core i5 11th Gen 1135G7 - (8 GB/5...   

                                                Images  \
0    https://rukminim2.flixcart.com/image/312/312/x...   
1    https://rukminim2.flixcart.com/image/312/312/x...   
2    https://rukminim2.flixcart.com/image/312/312/x...   
3    https://rukminim2.flixcart.com/image/312/312/x...   
4    https://

In [7]:
print(df.iloc[0][1])

https://rukminim2.flixcart.com/image/312/312/xif0q/computer/f/j/c/a324-51-thin-and-light-laptop-acer-original-imah42fxtvpkgryk.jpeg?q=70


C:\Users\manje\AppData\Local\Temp\ipykernel_276\134252878.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(df.iloc[0][1])


In [8]:
proj = pd.read_csv("flipkart_laptops.csv")
proj

,Unnamed: 0,Product Name,Images,Description,Original Price,Discounts,Discounted Price,Reviews
0,0,Acer Aspire 3 Intel Core i3 12th Gen 1215U - (...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'Intel Core i3 Processor (12th G...,"₹43,000",30% off,"₹29,990",4.0
1,1,HP 360 Intel Celeron Quad Core N4120 - (4 GB/6...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'Intel Celeron Quad Core Process...,"₹32,279",38% off,"₹19,990",4.0
2,2,ASUS Vivobook 15 Intel Core i3 12th Gen 1215U ...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'Intel Core i3 Processor (12th G...,"₹56,990",40% off,"₹33,990",4.2
3,3,Acer Swift Go 14 EVO OLED (2024) Intel Core i5...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'Intel Core i5 Processor (13th G...,"₹93,999",39% off,"₹56,990",4.3
4,4,Apple 2022 MacBook AIR Apple M2 - (8 GB/256 GB...,https://rukminim2.flixcart.com/image/312/312/x...,"{'Processor': 'Apple M2 Processor', 'RAM': '8 ...","₹99,900",5% off,"₹93,990",4.7
...,...,...,...,...,...,...,...,...
547,547,ASUS Vivobook S 15 OLED Intel Core i9 13th Gen...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'Intel Core i9 Processor (13th G...,"₹63,426",16% off,"₹94,990",4.2
548,548,HP 15q Intel Pentium Quad Core N5000 - (4 GB/1...,https://rukminim2.flixcart.com/image/312/312/j...,{'Processor': 'Intel Pentium Quad Core Process...,"₹66,990",26% off,"₹25,490",4.4
549,549,HP 247 G8 AMD Ryzen 5 Dual Core 3500U - (8 GB/...,https://rukminim2.flixcart.com/image/312/312/x...,{'Processor': 'AMD Ryzen 5 Dual Core Processor...,"₹67,000",NaN,"₹29,990",4.4
550,550,ASUS VivoBook 14 Intel Core i5 10th Gen 10210U...,https://rukminim2.flixcart.com/image/312/312/k...,{'Processor': 'Intel Core i5 Processor (10th G...,NaN,NaN,"₹55,990",NaN
